In [1]:
import torch
from transformers import AutoFeatureExtractor, Wav2Vec2ForPreTraining,get_scheduler
from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices
#from transformers.models.wav2vec2.feature_extraction_wav2vec2 import Wav2Vec2FeatureExtractor
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.optim import AdamW
import math

from model import Wav2Vec2ForPreTraining,Wav2Vec2Config,Wav2Vec2FeatureExtractor
from dataset import AudioDatasetSplits
from datasets import load_dataset


device = 'cuda'


In [2]:
#ds_train = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")


In [3]:
#feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")
feature_extractor = Wav2Vec2FeatureExtractor()
config = Wav2Vec2Config()
model = Wav2Vec2ForPreTraining(config).to(device)
ds_train = AudioDatasetSplits().train
#ds_val = AudioDatasetSplits().val

# ips = feature_extractor(ds_train[0]['audio'],return_tensors='pt',sampling_rate=ds_train[0]['sample_rate']).input_values
# batch_size, raw_sequence_length = ips.shape
# sequence_length = model._get_feat_extract_output_lengths(raw_sequence_length).item()

# mask_time_indices = _compute_mask_indices(
#     shape=(batch_size, sequence_length), mask_prob=model.config.mask_time_prob, mask_length=model.config.mask_time_length
# )

# sampled_negative_indices = _sample_negative_indices(
#     features_shape=(batch_size, sequence_length),
#     num_negatives=model.config.num_negatives,
#     mask_time_indices=mask_time_indices,
# )

# mask_time_indices = torch.tensor(data=mask_time_indices, device=ips.device, dtype=torch.long)
# sampled_negative_indices = torch.tensor(
#     data=sampled_negative_indices, device=ips.device, dtype=torch.long
# )

# model = model.train()
# loss = model(
#     ips, mask_time_indices=mask_time_indices, sampled_negative_indices=sampled_negative_indices
# )

In [4]:
def get_grad_norm(params, scale=1):
    """Compute grad norm given a gradient scale."""
    total_norm = 0.0
    for p in params:
        if p.grad is not None:
            param_norm = (p.grad.detach().data / scale).norm(2)
            total_norm += param_norm.item() ** 2
    total_norm = total_norm**0.5
    return total_norm

here we go

In [5]:
dl_train = DataLoader(ds_train,batch_size=8) # can put feature extractor in ds
#dl_val = DataLoader(ds_val,batch_size=8)   

In [6]:
next(iter(dl_train))['audio']

tensor([[ 1.3817e-06,  2.2483e-06,  1.8022e-06,  ...,  2.3067e-01,
          2.1962e-01,  2.2706e-01],
        [ 3.2028e-02,  5.8347e-02,  5.2269e-02,  ...,  8.1421e-02,
          1.1898e-01,  1.6805e-01],
        [ 5.0615e-11,  8.9776e-11,  6.9491e-11,  ..., -1.0592e-01,
         -1.2491e-01, -1.4430e-01],
        ...,
        [-7.9215e-08, -3.4389e-07, -3.7246e-07,  ..., -1.4127e-01,
         -2.1177e-01, -2.9916e-01],
        [ 1.3507e-07,  1.9804e-07,  3.3540e-07,  ...,  3.9985e-01,
          3.3562e-01,  2.8834e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  1.3280e-02,
          1.3537e-02,  1.5178e-02]])

In [9]:
gradient_accumulation_steps = 4
logging_steps = 10
max_gumbel_temperature = 2.0
min_gumbel_temperature = 0.5
gumbel_temperature_decay = 0.999995
num_train_epochs = 1
num_update_steps_per_epoch = math.ceil(len(dl_train) / gradient_accumulation_steps)

max_train_steps = num_train_epochs * num_update_steps_per_epoch

adam_beta1 = 0.9
adam_beta2 = 0.999
adam_epsilon = 1e-4
optimizer = AdamW(
        model.parameters(),
        lr=5e-4,
        betas=[adam_beta1, adam_beta2],
        eps=adam_epsilon,
    )


lr_scheduler = get_scheduler(
        name='linear',
        optimizer=optimizer,
        num_warmup_steps=110,
        num_training_steps=max_train_steps,
    )

In [10]:
start_epochs = 0
completed_steps = 0
progress_bar = tqdm(range(max_train_steps),disable=True)

for epoch in range(start_epochs,num_train_epochs):
    model.train()
    #batch_iterator = tqdm(dl_train,desc=f"Processing Epoch {epoch:02d}")
    for batch_idx,batch in enumerate(dl_train):

        batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}

      #  ips = feature_extractor(batch['audio']['array'],sampling_rate=16000,return_tensors='pt').input_values.squeeze(0).to(device)



       # print(batch['audio'].shape)
        ips = feature_extractor(batch['audio'],sampling_rate=batch['sample_rate'][0],return_tensors='pt').input_values.squeeze(0).to(device)
        #print(ips.shape)

        batch_size,raw_sequence_length = ips.shape

        sequence_length = model._get_feat_extract_output_lengths(raw_sequence_length).item()


        mask_time_indices = _compute_mask_indices(
            shape=(batch_size, sequence_length), mask_prob=model.config.mask_time_prob, mask_length=model.config.mask_time_length
        )

        sampled_negative_indices = _sample_negative_indices(
            features_shape=(batch_size, sequence_length),
            num_negatives=model.config.num_negatives,
            mask_time_indices=mask_time_indices,
        )

        mask_time_indices = torch.tensor(data=mask_time_indices, device=ips.device, dtype=torch.long)
        sampled_negative_indices = torch.tensor(
            data=sampled_negative_indices, device=ips.device, dtype=torch.long
        )

        #print(sampled_negative_indices.shape)

       #print(ips)
       # print(mask_time_indices)
       # print(sampled_negative_indices)

        outputs = model(ips,mask_time_indices=mask_time_indices,sampled_negative_indices=sampled_negative_indices)
        loss = outputs.loss

        loss.backward()

       # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        if (batch_idx + 1) % gradient_accumulation_steps == 0 or batch_idx == len(dl_train) - 1:
                grad_norm = get_grad_norm(model.parameters(), 1)


        optimizer.step()

        optimizer.zero_grad()

        lr_scheduler.step()

        gumbel_temperature = max(
                    max_gumbel_temperature * gumbel_temperature_decay**completed_steps,
                    min_gumbel_temperature,
                )
        
        model.set_gumbel_temperature(gumbel_temperature)

        completed_steps += 1

        # 6. Log all results
        if (batch_idx + 1) % (gradient_accumulation_steps * logging_steps) == 0:
            loss.detach()
            outputs.contrastive_loss.detach()
            outputs.diversity_loss.detach()

            train_logs = {
                "loss": (loss * gradient_accumulation_steps) ,
                "constrast_loss": outputs.contrastive_loss ,
                "div_loss": outputs.diversity_loss ,
                "ppl": outputs.codevector_perplexity,
                "lr": torch.tensor(optimizer.param_groups[0]["lr"]),
                "temp": torch.tensor(gumbel_temperature),
                "grad_norm": torch.tensor(grad_norm),
            }
            log_str = ""
            for k, v in train_logs.items():
                log_str += "| {}: {:.3e}".format(k, v.item())

            progress_bar.write(log_str)

            
        

        




        #print('loss',loss)

   


        #batch_iterator.set_postfix(loss=running_loss /( 1 if batch_idx == 0 else batch_idx))

| loss: 1.478e+03| constrast_loss: 3.695e+02| div_loss: 1.676e-01| ppl: 6.387e+02| lr: 1.818e-04| temp: 2.000e+00| grad_norm: 5.689e+01
| loss: 2.963e+03| constrast_loss: 7.408e+02| div_loss: 3.909e-01| ppl: 6.384e+02| lr: 3.636e-04| temp: 1.999e+00| grad_norm: 1.040e+02
| loss: 2.911e+03| constrast_loss: 7.277e+02| div_loss: 3.884e-01| ppl: 6.384e+02| lr: 4.820e-04| temp: 1.999e+00| grad_norm: 1.313e+02
| loss: 2.962e+03| constrast_loss: 7.404e+02| div_loss: 2.032e-01| ppl: 6.392e+02| lr: 4.101e-04| temp: 1.998e+00| grad_norm: 8.022e+01
| loss: 2.961e+03| constrast_loss: 7.403e+02| div_loss: 1.280e-01| ppl: 6.395e+02| lr: 3.381e-04| temp: 1.998e+00| grad_norm: 8.202e+01
| loss: 2.968e+03| constrast_loss: 7.420e+02| div_loss: 1.232e-01| ppl: 6.395e+02| lr: 2.662e-04| temp: 1.998e+00| grad_norm: 1.190e+02
| loss: 2.922e+03| constrast_loss: 7.304e+02| div_loss: 6.827e-02| ppl: 6.397e+02| lr: 1.942e-04| temp: 1.997e+00| grad_norm: 6.009e+01
| loss: 2.754e+03| constrast_loss: 6.886e+02| di

KeyboardInterrupt: 

In [ ]:
audio = 1, 80000
tensor([[1.3817e-06, 2.2483e-06, 1.8022e-06,  ..., 2.3067e-01, 2.1962e-01,
         2.2706e-01]], device='cuda:0')
------------------------------------------------------------------

ips = 1, 80000 
tensor([[2.2682e-04, 2.3256e-04, 2.2961e-04,  ..., 1.5293e+00, 1.4560e+00,
         1.5053e+00]], device='cuda:0')
------------------------------------------------------------------

extract_features = 1,249,512
tensor([[[ 0.1235,  1.0637,  0.3155,  ...,  0.1192, -0.0555,  0.3207],
         [-0.0291, -0.0841, -0.1173,  ..., -0.0075,  0.0098, -0.0437],
         [ 0.1854, -0.1252,  0.4169,  ...,  0.0302, -0.1354,  0.0635],
         ...,
         [-0.0499, -0.1683, -0.0749,  ..., -0.0380,  0.0608,  0.0732],
         [ 0.1800,  0.1400,  2.0366,  ...,  0.0052,  0.1235,  0.0312],
         [ 0.0764,  0.1193,  0.6772,  ...,  0.0497,  0.1072,  0.0665]]],
       device='cuda:0', grad_fn=<GeluBackward0>)
------------------------------------------------------------------
hidden_states = 1,249,768
tensor([[[ 0.0339, -0.4830, -0.0305,  ...,  0.0159,  0.0779, -0.2026],
         [ 0.2883, -0.7021,  0.6997,  ...,  0.5885,  0.3514,  0.1212],
         [-0.2194, -0.4437,  0.2657,  ..., -0.2675, -0.3728, -0.4760],
         ...,
         [-0.4071, -0.5601, -0.1708,  ..., -0.6891, -0.8761,  0.1655],
         [-0.2948, -0.3244,  0.1004,  ...,  0.0792,  0.1473, -0.1143],
         [-0.0085,  0.3465,  0.3159,  ...,  0.7847, -0.2695,  0.7689]]],
       device='cuda:0', grad_fn=<ViewBackward0>) 
------------------------------------------------------------------
extract_features = 1,249,512
tensor([[[-0.9377, -0.0026,  0.2004,  ...,  1.1388, -1.0242,  0.4582],
         [-0.8192, -0.6528, -0.4071,  ..., -0.6667,  0.5033, -0.5041],
         [-0.3353,  0.6557,  1.2236,  ..., -0.5683,  0.2599,  0.5061],
         ...,
         [ 0.2780, -0.0862,  0.7484,  ..., -0.8934,  1.6097, -0.2075],
         [-1.7482,  0.5112,  0.4290,  ...,  1.2316, -0.8526, -0.3602],
         [-0.2306,  0.0931,  0.1837,  ...,  0.0942, -0.6955, -1.0463]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>) 
------------------------------------------------------------------

hidden_states = 1,249, 768
tensor([[[-0.1991,  0.3561,  0.1502,  ...,  0.0869,  0.6993,  0.1601],
         [ 0.9603,  0.2272, -0.1970,  ...,  0.2197,  0.0703, -0.3534],
         [-0.0289,  0.0383, -0.1532,  ...,  0.4002,  0.6341, -0.0135],
         ...,
         [ 0.0797, -0.0814,  0.6674,  ...,  0.1392,  0.4300, -0.2692],
         [-0.3952, -0.6119, -0.5780,  ...,  0.7492,  0.2148,  0.2660],
         [ 0.7736, -0.1803, -0.1786,  ..., -0.3715, -0.1962, -0.0951]]],
------------------------------------------------------------------

encoder_outputs (last hidden state) = [1, 249, 768])
tensor([[[ 1.8529,  1.2018,  1.2603,  ..., -0.4005, -0.7679,  1.5168],
         [ 2.4584,  0.3279,  1.1077,  ...,  0.0198,  0.5620,  1.3509],
         [ 0.8397,  1.1146,  1.2582,  ...,  0.7337,  0.1549,  0.1386],
         ...,
         [ 2.3995,  0.0749,  0.3688,  ...,  0.1632, -0.3292,  0.8795],
         [ 2.0309,  1.2241,  2.5985,  ..., -0.2022,  1.0152,  0.7052],
         [-0.0648,  0.1555,  0.9045,  ...,  0.6808,  1.3329, -1.1029]]],
------------------------------------------------------------------ <- pretraining model

transformer_features = 1, 249, 256
tensor([[[ 0.4395,  0.0093,  0.1106,  ...,  0.1624, -0.4259, -0.1438],
         [ 0.2835,  0.0836, -0.1176,  ..., -1.4001, -0.6082, -0.4137],
         [ 0.1038, -0.6754,  0.0953,  ...,  0.0870, -0.5074, -0.2045],
         ...,
         [ 0.7046, -0.7818, -0.0214,  ...,  0.1316, -0.0481, -0.3063],
         [-0.4119,  0.0397,  0.2056,  ...,  0.2657,  0.0776,  0.1673],
         [ 0.5931, -0.5297,  0.2475,  ..., -1.0861, -0.2295,  1.0191]]],
       device='cuda:0', grad_fn=<ViewBackward0>) 
------------------------------------------------------------------

extract_features = 1, 249, 512
tensor([[[-0.5587, -1.2472, -1.0980,  ...,  0.8418, -0.5749,  0.4490],
         [-0.7627, -0.4484, -0.7772,  ..., -0.2958,  1.2044, -0.6321],
         [-0.8375, -0.0000, -0.9049,  ..., -0.9059,  0.1687, -0.6254],
         ...,
         [-0.0000, -0.0000, -2.3028,  ..., -0.5176,  3.0487, -0.0000],
         [-1.0692, -0.0000, -0.8404,  ..., -0.3041,  1.6150, -0.0000],
         [-0.0000,  0.0000, -0.4952,  ..., -0.0000,  2.0064,  0.7426]]],
------------------------------------------------------------------

quantized_features 1,249,256
tensor([[[-0.0007,  0.0041,  0.0038,  ..., -0.0010,  0.0016,  0.0003],
         [ 0.0035,  0.0055,  0.0023,  ...,  0.0107, -0.0053, -0.0056],
         [-0.0015,  0.0167, -0.0020,  ..., -0.0018, -0.0030, -0.0007],
         ...,
         [ 0.0006, -0.0015, -0.0092,  ..., -0.0038,  0.0029,  0.0095],
         [ 0.0033,  0.0141,  0.0015,  ..., -0.0038, -0.0048, -0.0006],
         [ 0.0003, -0.0084, -0.0059,  ..., -0.0030, -0.0111, -0.0071]]]

------------------------------------------------------------------<- into quantitation module

hidden_states
tensor([[[ 0.6000, -0.6655,  0.4842,  ..., -0.3797,  0.4714, -1.1692],
         [ 0.2132, -0.8672,  0.2732,  ...,  0.2052,  0.5152,  0.3293],
         [ 0.2438, -0.2918,  0.5199,  ..., -0.3120,  0.3674, -0.1998],
         ...,
         [ 0.7111,  0.0052, -0.3853,  ...,  0.0501, -0.6594,  0.5257],
         [ 1.0093,  0.3563,  0.7620,  ..., -0.9669,  0.4358, -0.4640],
         [ 0.8773, -0.3935, -0.9076,  ..., -0.9158,  0.4923,  0.1731]]],
       device='cuda:0', grad_fn=<ViewBackward0>) torch.Size([1, 249, 640]) 
      
tensor([[ 0.6000, -0.6655,  0.4842,  ...,  0.2033, -0.2749,  1.0719],<- then reshaped to this
        [ 0.3865, -0.2972,  0.2173,  ..., -0.3797,  0.4714, -1.1692],
        [ 0.2132, -0.8672,  0.2732,  ...,  0.4031,  0.8485,  0.5515],
        ...,
        [-0.1745,  0.4993, -0.1358,  ..., -0.9669,  0.4358, -0.4640],
        [ 0.8773, -0.3935, -0.9076,  ...,  0.2689,  0.0802, -0.2527],
        [ 0.1804,  0.3172, -0.1329,  ..., -0.9158,  0.4923,  0.1731]],
       device='cuda:0', grad_fn=<ViewBackward0>) torch.Size([498, 320])

------------------------------------------------------------------<- 
codevector_probs 498, 320

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]


SyntaxError: invalid syntax (3846610266.py, line 4)